<a href="https://colab.research.google.com/github/da-head0/youreditor/blob/main/5_subtitles_to_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import glob

glob.glob('/content/drive/MyDrive/dataset/subs/lln_excel_subs_2021-4-25_*.xlsx')

In [ ]:
all_data = pd.DataFrame()
for f in glob.glob('/content/drive/MyDrive/dataset/subs/lln_excel_subs_2021-4-25_*.xlsx'):
    df = pd.read_excel(f)
    all_data = all_data.append(df,ignore_index=True)

all_data.head()

,Subtitle,Translation
0,"- THAT WOULD BE FOUR, ACTUALLY.",세 번 연속으로 이기셨네요 정확히는 네 번 연속이에요
1,"- OH, GOD. - COME ON.",정확히는 네 번 연속이에요
2,"WHAT DO YOU SAY, WE GO FIVE OUTTA NINE?",이제 9판 5승제로 갈까요?
3,"- SO YOU CAN KEEP ABUSING ME? NO, THANK YOU. I...","계속 갖고 노시려고요?\n됐어요, 그만할래요"
4,"- OH, COME ON. I'LL SPOT YOU THREE.",세 판 핸디캡 드릴게요


In [ ]:
all_data.shape

(34090, 2)

- 생각보다 별로 안되네...

In [ ]:
all_data.loc[all_data['Subtitle']=='']

,Subtitle,Translation
1,"- OH, GOD. - COME ON.",정확히는 네 번 연속이에요


In [ ]:
import re
# 화면자막 지우기
text = '[SMILE] (한숨) "화면자막" Hey! Whats up?'
text = re.sub(r'\([^)]*\)','',text).strip() # () 안 지시문 없애기
text = re.sub(r"\[.+\]",'',text).strip() # [] 안 지시문 없애기
text = re.sub(r"\".+\"",'',text).strip() # "화면자막" 없애기
text

'화면자막 Hey! Whats up?'

# Todo
- 숫자 속의 , 양 옆의 빈칸 없애기

In [ ]:
text = "that's 4 , 140 pages . plus all this"
text = re.sub(r'[\W]','',text).strip()
text

'thats4140pagesplusallthis'

In [ ]:
w = '["너는 무슨 역할을 하는 거니?"]'
w = re.sub(r'[" "]+', " ", w) # "를 지우고 한칸 띄워줌 - "" 안의 문자까지 없애진 않는다.
w

'[ 너는 무슨 역할을 하는 거니? ]'

In [ ]:
all_data['Subtitle'][19]

'- YOU MEAN THIS? - ( sighs )'

- 이런 지시문도 잡아주어야 한다.

In [ ]:
import unicodedata
# 유니코드 파일을 아스키 코드 파일로 변환합니다.
def unicode_to_ascii(s):
  return ''.join(c for c in unicodedata.normalize('NFD', s)
      if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())

  # 단어와 단어 뒤에 오는 구두점(.)사이에 공백을 생성합니다.
  # 예시: "he is a boy." => "he is a boy ."
  # 참고:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"\[.+\]",'',w).strip() # 영어 문장 속의 [smile] 같은 지시문 없애기
  w = re.sub(r'\([^)]*\)','',w).strip() # (sigh) 같은 지시문 없애기
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  #w = re.sub(r" . . .", r" ...", w) # . . . -> ... 
  w = re.sub(r'[" "]+', " ", w) # "를 지우고 한칸 띄워줌 - 혹시나 있을 "를 공백으로 치환.

  # (a-z, A-Z, ".", "?", "!", ",")을 제외한 모든 것을 공백으로 대체합니다.
  w = re.sub(r"[^0-9a-zA-Z?.!',¿$]+", " ", w) # ' 포함 - 숫자 빼면 안 된다!!

  w = w.strip()

  # 모델이 예측을 시작하거나 중단할 때를 알게 하기 위해서
  # 문장에 start와 end 토큰을 추가합니다.
  #w = '<start> ' + w + ' <end>'
  return w


def preprocess_sentence_kr(w):
  w = w.strip()

  # 단어와 단어 뒤에 오는 구두점(.)사이에 공백을 생성합니다.
  # 예시: "he is a boy." => "he is a boy ."
  # 참고:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"\".+\"",'',w).strip() # 큰따옴표 제거 - 화면자막
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  #w = re.sub(r" . . .", r" ...", w) # 이거 넣으니 한국어에서 짤리는 문장이 발생
  w = re.sub(r'[" "]+', " ", w)

  # (a-z, A-Z, ".", "?", "!", ",")을 제외한 모든 것을 공백으로 대체합니다.
  w = re.sub(r"[^0-9가-힣?.!,¿]+", " ", w) # \n도 공백으로 대체해줌
  #w = re.sub(r'[ |ㄱ-ㅎ|ㅏ-ㅣ]+', " ", w)

  w = w.strip()

  # 모델이 예측을 시작하거나 중단할 때를 알게 하기 위해서
  # 문장에 start와 end 토큰을 추가합니다.
  #w = '<start> ' + w + ' <end>'
  return w

In [ ]:
df = pd.DataFrame()

In [ ]:
df['en'] = [preprocess_sentence(l) for l in all_data['Subtitle']]
df['kr'] = [preprocess_sentence_kr(l) for l in all_data['Translation']]

In [ ]:
df

,en,kr
0,"that would be four , actually .",세 번 연속으로 이기셨네요 정확히는 네 번 연속이에요
1,"oh , god . come on .",정확히는 네 번 연속이에요
2,"what do you say , we go five outta nine ?",이제 9판 5승제로 갈까요 ?
3,"so you can keep abusing me ? no , thank you . ...","계속 갖고 노시려고요 ? 됐어요 , 그만할래요"
4,"oh , come on . i'll spot you three .",세 판 핸디캡 드릴게요
...,...,...
34085,report card . mm hmm . report card .,성적표
34086,notice from the city to take down the tree hou...,나무 위 집을 계속 놔둘 거면
34087,or be fined $25 a day .,하루 25달러씩 벌금 내라는 편지요
34088,report card .,성적표


- 와우 화면 자막도 있네....

In [ ]:
df.iloc[196]

en    the bigger they are , the harder they fall .
kr                     몸집이 클수록 처참하게 무너지지 휴대폰 소지 금지
Name: 196, dtype: object

- "휴대폰 소지 금지" 로 들어가면 모를까 '' 로 감싸이면 '강조표시' 랑 구분이 안되어 지울 수가 없다... ^^....

- 하.. 일단 무시...

In [ ]:
for i in range(0,100):
  print(all_data.iloc[i,0]) # en
  print(all_data.iloc[i,1]) # kr
  print('-----')

- THAT WOULD BE FOUR, ACTUALLY.
세 번 연속으로 이기셨네요 정확히는 네 번 연속이에요
-----
- OH, GOD. - COME ON.
정확히는 네 번 연속이에요
-----
WHAT DO YOU SAY, WE GO FIVE OUTTA NINE?
이제 9판 5승제로 갈까요?
-----
- SO YOU CAN KEEP ABUSING ME? NO, THANK YOU. I'M DONE.
계속 갖고 노시려고요?
됐어요, 그만할래요
-----
- OH, COME ON. I'LL SPOT YOU THREE.
세 판 핸디캡 드릴게요
-----
- GAME TIME'S OVER, WYATT. THEY'RE HERE. - THEY'RE HERE?
-와이엇 씨, 그만 놀고 가시죠
-왔나요?
-----
- YOU CAN SET UP IN THE CONFERENCE ROOM. - YEAH. NO, I'M JUST...
-회의실 가서 준비하도록 하세요
-아뇨, 저기...
-----
- GET SET UP IN THE CONFERENCE ROOM.
회의실 가셔야죠
-----
YOU DID LET HIM WIN, RIGHT? - IT WASN'T EASY.
-져준 거 맞지?
-쉽지 않았어요
-----
THE GUY HAS THE COORDINATION OF A TWO-YEAR-OLD.
2살짜리 같더라고요
-----
- I KNOW. THE LAST TIME THESE VENTURE CAPITALISTS CAME IN,
지난번엔 복싱했었어
-----
I TOOK HIM BOXING TO CALM HIM DOWN.
지난번엔 복싱했었어 이기게 해 주려고
내 얼굴을 때리다시피 했다니까
-----
I BASICALLY HAD TO PUNCH MYSELF IN THE FACE TO LET HIM WIN.
이기게 해 주려고
내 얼굴을 때리다시피 했다니까
-----
I DON'T GET IT. I MEAN, IF HE'S SUCH A GENIUS, WHY DOES HE GET 

- 노가다로 화면자막을 잡아주어야 하나?
영문 에피소드 제목명이 드라마 제목명으로 pair가 된 것을 발견.

♪ I'VE GOT TO DANCE ♪
'슈츠'

In [ ]:
# 한국어는 안 합치는 버전

#lines_seen = set() # holds lines already seen
previous = "세 번 연속으로 이기셨네요 정확히는 네 번 연속이에요"

for i in range(1,len(all_data)):
  #print(df.iloc[i,0]) # en

  # 이전 문장이 현재 문장과 중복되는 부분이 있으면(뒷 문장이 더 김)
  if previous in df.iloc[i,1]:
  #if previous in df.loc[i,'kr']:    # 오류날 땐 이 버전으로 한번 돌리고 다시 iloc으로 돌리면 된다...
    df.iloc[i,0] = df.iloc[i-1,0] + ' '+ df.iloc[i,0] # 뒷 문장에 내려서 합치고
    df = df.drop(df.index[i-1]) # 이전 문장을 지운다
    previous = df.iloc[i,1]
  
  # 현재 문장이 이전 문장과 중복되는 부분이 있으면 (이전 문장이 더 김)
  if df.iloc[i,1] in previous:
  #if df.loc[i,'kr'] in previous:    
    df.iloc[i-1,0] = df.iloc[i-1,0] + ' '+ df.iloc[i,0] # 이전 문장에 올려서 합치고
    df = df.drop(df.index[i]) # 현재 문장을 지운다
    previous = df.iloc[i-1,1]

  else: # 중복이 아니면
    previous = df.iloc[i,1] # 현재 문장을 previous 로 만들고 루프 돌리기

  if i>len(df):
    print(f"{i-1} 줄까지 실행 완료, 한국어 문장 = {df.iloc[i-1,1]}")
    break

- iloc 안 될땐 loc으로 고치고 에러날때 다시 iloc 돌리면 된다..

- 왜 오류가 나냐면, 한번 돌리면 중복 문장이 줄어들고 df 길이가 바뀌는데, 그러면 인덱스 번호가 바뀌기 때문이다.
reset_index를 같이 해줘야 하나??


무튼 오류가 나도 어느 정도 돌아간 상태이니 df.tail() 부분 확인하고, 괜찮으면 ok

In [ ]:
df[:50]

,en,kr
0,"that would be four , actually . oh , god . com...",세 번 연속으로 이기셨네요 정확히는 네 번 연속이에요
2,"what do you say , we go five outta nine ?",이제 9판 5승제로 갈까요 ?
3,"so you can keep abusing me ? no , thank you . ...","계속 갖고 노시려고요 ? 됐어요 , 그만할래요"
4,"oh , come on . i'll spot you three .",세 판 핸디캡 드릴게요
5,"game time's over , wyatt . they're here . they...","와이엇 씨 , 그만 놀고 가시죠 왔나요 ?"
6,you can set up in the conference room . yeah ....,"회의실 가서 준비하도록 하세요 아뇨 , 저기 . . ."
7,get set up in the conference room .,회의실 가셔야죠
8,"you did let him win , right ? it wasn't easy .",져준 거 맞지 ? 쉽지 않았어요
9,the guy has the coordination of a two year old .,2살짜리 같더라고요
11,i know . the last time these venture capitalis...,지난번엔 복싱했었어 이기게 해 주려고 내 얼굴을 때리다시피 했다니까


In [ ]:
df.tail(50)

,en,kr
34035,okay .,그래
34036,i m going to get something for my stomach .,먹을 것 좀 사 올게요
34037,"come on , swipe .",제발 긁혀라
34038,swipe !,긁혀라 !
34039,"mitchell , i need your help .","미첼 , 좀 도와줘"
34040,"it won t swipe , and i m in no condition to se...","카드가 안 긁혀 , 나 지금 직원 상대할 기분 아니거든"
34042,mitchell . mitchell ? mitchell ? mitch mitchell !,미첼
34044,where are you going ?,어디 가 ?
34045,"i m just sick of it , cam .",지긋지긋해
34047,"mitchell , come back here and talk to me . the...","미첼 , 이리 와서 나랑 얘기해 여긴 코요테가 있어 마약 중독자도 !"


In [ ]:
df.loc[34089,'en'] # 이렇게 해도 저 인덱스 번호로 호출 가능. iloc은 안됨.

'red light ticket from when me and manny stole that'

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.shape

(29863, 2)

- 4천개가 주는 매직~~~

## TODO
[] 화면자막 지우기

- ...으로 끝나는 문장들 뒤랑 붙여줘야 할 수도 있음

In [ ]:
df.to_csv('/content/drive/MyDrive/dataset/corpus/netsubs1_0426.csv', index=False)

- 드라이브에 저장, 잘 저장됐는지 확인

In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/dataset/netsubs.csv')

In [ ]:
df2

,en,kr
0,"that would be four , actually . oh , god . com...",세 번 연속으로 이기셨네요 정확히는 네 번 연속이에요
1,"what do you say , we go five outta nine ?",이제 9판 5승제로 갈까요 ?
2,"so you can keep abusing me ? no , thank you . ...","계속 갖고 노시려고요 ? 됐어요 , 그만할래요"
3,"oh , come on . i'll spot you three .",세 판 핸디캡 드릴게요
4,"game time's over , wyatt . they're here . they...","와이엇 씨 , 그만 놀고 가시죠 왔나요 ?"
...,...,...
30250,report card . mm hmm . report card .,성적표
30251,notice from the city to take down the tree hou...,나무 위 집을 계속 놔둘 거면
30252,or be fined $25 a day .,하루 25달러씩 벌금 내라는 편지요
30253,report card .,성적표
